# Ticino Covid - Data Analysis and modelling


In [1]:
# Import libraries
import pandas as pd
import numpy as np

## Loading data

In [3]:
# Load dataset using pandas
df = pd.read_csv('https://raw.githubusercontent.com/openZH/covid_19/master/fallzahlen_kanton_total_csv_v2/COVID19_Fallzahlen_Kanton_TI_total.csv')
df

,data,stato,codice_regione,denominazione_regione,codice_provincia,denominazione_provincia,sigla_provincia,lat,long,totale_casi,note,codice_nuts_1,codice_nuts_2,codice_nuts_3
0,2020-02-24T18:00:00,ITA,13,Abruzzo,66,L'Aquila,AQ,42.351222,13.398438,0,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,13,Abruzzo,67,Teramo,TE,42.658918,13.704400,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,13,Abruzzo,68,Pescara,PE,42.464584,14.213648,0,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,13,Abruzzo,69,Chieti,CH,42.351032,14.167546,0,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,13,Abruzzo,979,In fase di definizione/aggiornamento,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200222,2023-11-15T17:00:00,ITA,5,Veneto,27,Venezia,VE,45.434905,12.338452,471260,NaN,ITH,ITH3,ITH35
200223,2023-11-15T17:00:00,ITA,5,Veneto,28,Padova,PD,45.406930,11.876087,552303,NaN,ITH,ITH3,ITH36
200224,2023-11-15T17:00:00,ITA,5,Veneto,29,Rovigo,RO,45.071073,11.790070,124342,NaN,ITH,ITH3,ITH37
200225,2023-11-15T17:00:00,ITA,5,Veneto,899,Fuori Regione / Provincia Autonoma,NaN,NaN,NaN,39850,NaN,ITH,ITH3,NaN
